# 二手房总价预测模型

In [1]:
import pandas as pd
import numpy as np
import copy
from sklearn.model_selection  import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV

## 模型建立

### 1.1读取文件

In [2]:
d1=pd.read_csv("wuhan_houses_clean.csv",encoding='utf-8')
d1.head()

,户型,建筑面积,朝向,楼层,装修,建筑年代,电梯,产权性质,住宅类别,建筑结构,建筑类别,区域,单价,地铁,地铁距离,总价
0,2室1厅1卫,64.48,东南,低层,精装修,2006,有,商品房,普通住宅,平层,板楼,硚口,13958.0,1号线,587米,90.0
1,2室2厅1卫,64.10,西南,中层,精装修,2009,有,商品房,普通住宅,平层,板楼,江岸,16069.0,8号线,946米,103.0
2,4室2厅2卫,148.05,南北,中层,精装修,2006,有,商品房,普通住宅,平层,板楼,武昌,17224.0,7号线,509米,255.0
3,3室2厅1卫,110.39,东南,低层,精装修,2006,有,商品房,普通住宅,平层,板楼,武昌,19023.0,7号线,509米,210.0
4,3室2厅2卫,129.16,南北,低层,中装修,2005,有,商品房,普通住宅,平层,板楼,武昌,15485.0,7号线,613米,200.0


### 1.2特征值预处理

In [3]:
#删除所有的缺失值
d1=d1.dropna().reset_index(drop=True)

In [4]:
#将户型分解，2室1厅1卫拆分为room:2,hall:1,wc:1
d1["室"]=d1["户型"].map(lambda x: x.split("室")[0])
d1["厅"]=d1["户型"].map(lambda x: x.split("厅")[0].split("室")[1])
d1["卫"]=d1["户型"].map(lambda x: x.split("卫")[0].split("厅")[1])
d1.head(3)

,户型,建筑面积,朝向,楼层,装修,建筑年代,电梯,产权性质,住宅类别,建筑结构,建筑类别,区域,单价,地铁,地铁距离,总价,室,厅,卫
0,2室1厅1卫,64.48,东南,低层,精装修,2006,有,商品房,普通住宅,平层,板楼,硚口,13958.0,1号线,587米,90.0,2,1,1
1,2室2厅1卫,64.10,西南,中层,精装修,2009,有,商品房,普通住宅,平层,板楼,江岸,16069.0,8号线,946米,103.0,2,2,1
2,4室2厅2卫,148.05,南北,中层,精装修,2006,有,商品房,普通住宅,平层,板楼,武昌,17224.0,7号线,509米,255.0,4,2,2


In [5]:
#删掉没用的列，户型，单价
d1.drop(columns=["户型","单价"],inplace=True)

In [6]:
d1.head()

,建筑面积,朝向,楼层,装修,建筑年代,电梯,产权性质,住宅类别,建筑结构,建筑类别,区域,地铁,地铁距离,总价,室,厅,卫
0,64.48,东南,低层,精装修,2006,有,商品房,普通住宅,平层,板楼,硚口,1号线,587米,90.0,2,1,1
1,64.10,西南,中层,精装修,2009,有,商品房,普通住宅,平层,板楼,江岸,8号线,946米,103.0,2,2,1
2,148.05,南北,中层,精装修,2006,有,商品房,普通住宅,平层,板楼,武昌,7号线,509米,255.0,4,2,2
3,110.39,东南,低层,精装修,2006,有,商品房,普通住宅,平层,板楼,武昌,7号线,509米,210.0,3,2,1
4,129.16,南北,低层,中装修,2005,有,商品房,普通住宅,平层,板楼,武昌,7号线,613米,200.0,3,2,2


### 1.3编码

编码-有序多分类，按照对总价影响程度排名，越大影响越高(由powerbi可视化结果)

无序多分类无法直接引入，必须“哑元”化变量

等级变量（有序多分类）可以直接引入模型

In [7]:
df=copy.deepcopy(d1)
df.head(2)

,建筑面积,朝向,楼层,装修,建筑年代,电梯,产权性质,住宅类别,建筑结构,建筑类别,区域,地铁,地铁距离,总价,室,厅,卫
0,64.48,东南,低层,精装修,2006,有,商品房,普通住宅,平层,板楼,硚口,1号线,587米,90.0,2,1,1
1,64.10,西南,中层,精装修,2009,有,商品房,普通住宅,平层,板楼,江岸,8号线,946米,103.0,2,2,1


In [8]:
df.columns

Index(['建筑面积', '朝向', '楼层', '装修', '建筑年代', '电梯', '产权性质', '住宅类别', '建筑结构', '建筑类别',
       '区域', '地铁', '地铁距离', '总价', '室', '厅', '卫'],
      dtype='object')

In [9]:
#朝向
map1={'东南':8, '西南':9, '南北':6, '南':4, '西':1, '东北':3, '北':2, '东':5, '西北':10, '东西':7}
df["朝向"]=df["朝向"].map(map1)

In [10]:
#楼层
map2={'低层':1, '中层':3, '高层':2, '底层':4, '顶层':5}
df["楼层"]=df["楼层"].map(map2)

In [11]:
#装修
map3={'精装修':4, '中装修':2, '毛坯':3, '简装修':1, '豪华装修':5}
df["装修"]=df["装修"].map(map3)

In [12]:
#电梯
map4={'有':1, '无':2}
df["电梯"]=df["电梯"].map(map4)

In [13]:
#产权性质
map5={'商品房':2, '经济适用房':1}
df["产权性质"]=df["产权性质"].map(map5)

In [14]:
#住宅类别
map6={'普通住宅':4, '平层':1, '公寓':2, '经济适用房':3}
df["住宅类别"]=df["住宅类别"].map(map6)

In [15]:
#建筑结构
map7={'平层':1, '复式':2}
df["建筑结构"]=df["建筑结构"].map(map7)

In [16]:
#建筑类别
map8={'板楼':1}
df["建筑类别"]=df["建筑类别"].map(map8)

In [17]:
#区域
map9={'硚口':11, '江岸':15, '武昌':14, '东湖高新区':12, '洪山':8, '汉阳':9, '青山':10, '江汉':13, '东西湖':5, '江夏':4,
       '武汉周边':1, '沌口':6, '黄陂':3, '经济开发区':5, '汉南':7, '蔡甸':2}
df["区域"]=df["区域"].map(map9)

In [18]:
#地铁
map10={'1号线':14, '8号线':11, '7号线':8, '无':4, '5号线':9, '19号线':3, '11号线':6, '2号线':10, '6号线':5,
       '4号线':13, '3号线':12, '21号线':2, '16号线':1}
df["地铁"]=df["地铁"].map(map10)

In [19]:
#地铁距离
import re
#去掉最后的单位，并且变为int
def clean_subway_distance(distance):
    if distance != "无":
        try:
            # 去掉非数字字符并转换为整数
            return int(re.sub(r'[^0-9]', '', distance))
        except ValueError:
            return 0  # 如果转换失败，设为默认值
    else:
        return 0  # 如果没有地铁距离，设为默认值

# 应用处理函数
df["地铁距离"] = df["地铁距离"].apply(clean_subway_distance)

In [20]:
#建筑年代
#改为房龄，变为int
df["建筑年代"]=df["建筑年代"].astype('int32')
df["房龄"]=d1["建筑年代"].map(lambda x : 2025-x)
df.drop(columns=["建筑年代"],inplace=True)

In [21]:
df['室'] = df['室'].astype(int)
df['厅'] = df['厅'].astype(int)
df['卫'] = df['卫'].astype(int)

In [22]:
df.head()

,建筑面积,朝向,楼层,装修,电梯,产权性质,住宅类别,建筑结构,建筑类别,区域,地铁,地铁距离,总价,室,厅,卫,房龄
0,64.48,8,1,4,1,2,4,1,1,11,14,587,90.0,2,1,1,19
1,64.10,9,3,4,1,2,4,1,1,15,11,946,103.0,2,2,1,16
2,148.05,6,3,4,1,2,4,1,1,14,8,509,255.0,4,2,2,19
3,110.39,8,1,4,1,2,4,1,1,14,8,509,210.0,3,2,1,19
4,129.16,6,1,2,1,2,4,1,1,14,8,613,200.0,3,2,2,20


### 1.4模型建立

In [23]:
from sklearn.metrics import r2_score

In [24]:
X = df.drop(columns=["总价"])
y=df["总价"]
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=46)
poly=PolynomialFeatures(degree=2)
x_train = poly.fit_transform(X_train)
x_test=poly.fit_transform(X_test)

In [26]:
#随机森林
rf = RandomForestRegressor()
rf.fit(x_train,y_train)
y_pred = rf.predict(x_test)
r2 = r2_score(y_test, y_pred)
print(f"训练集得分：{round(rf.score(x_train,y_train),2)}")
print(f"测试集集得分：{round(rf.score(x_test,y_test),2)}")
print(f"R^2: {r2:.4f}")

训练集得分：0.98
测试集集得分：0.83
R^2: 0.8327


In [27]:
#决策树
dt=DecisionTreeRegressor(max_depth=6)
dt.fit(x_train,y_train)
y_pred = dt.predict(x_test)
r2 = r2_score(y_test, y_pred)
print(f"训练集得分：{round(dt.score(x_train,y_train),2)}")
print(f"测试集集得分：{round(dt.score(x_test,y_test),2)}")
print(f"R^2: {r2:.4f}")

训练集得分：0.88
测试集集得分：0.76
R^2: 0.7642


In [28]:
#k近邻
kn=KNeighborsRegressor(n_neighbors=7)
kn.fit(x_train,y_train)
y_pred = kn.predict(x_test)
r2 = r2_score(y_test, y_pred)
print(f"训练集得分：{round(kn.score(x_train,y_train),2)}")
print(f"测试集集得分：{round(kn.score(x_test,y_test),2)}")
print(f"R^2: {r2:.4f}")

训练集得分：0.83
测试集集得分：0.76
R^2: 0.7610


比较上面集中模型，最终测试集的得分都能保持在75%以上。

其中随机森林训练集得分达到80%以上，是表现最好的，所以选择对随机森林进行调参优化·

### 1.5模型调参(随机森林)

随机森林训练集得分0.98，测试集仅0.83，说明模型过于复杂

#### 1.5.1去掉多项式特征

因为上面使用了PolynomialFeatures(degree=2)生成了大量多项交叉特征，其实会使树模型过拟合

所以这里去掉，使用原始的特征

In [30]:
# 删除 PolynomialFeatures 步骤，直接使用原始特征
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=46)

rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f"训练集得分：{rf.score(X_train, y_train):.4f}")
print(f"无多项式特征的测试集得分：{rf.score(X_test, y_test):.4f}")
print(f"R^2: {r2:.4f}")

训练集得分：0.9809
无多项式特征的测试集得分：0.8618
R^2: 0.8618


## 1.6实际预测

有一家三口，孩子即将上学，大人在汉阳工作，需要购买房子，要求如下：

三室一厅一卫(3、1、1)

面积大概90平米左右(90)

中层(3)

靠近地铁(7)

东南(8)

简装修(1)

电梯(1)

商品房(2)

普通住宅(4)

板楼(1)

汉阳(9)

距离地铁的大概距离(900)

房龄(20)

In [203]:
apply = np.array([90, 8, 3, 1, 1, 1, 1, 1, 1, 9, 6, 900, 3, 1, 1, 20]).reshape(1, -1)

In [204]:
apply_df = pd.DataFrame(apply, columns=X_train.columns) 
poly_apply = poly.transform(apply_df) 
original_apply = apply_df  

In [205]:
dt_pred=round(dt.predict(poly_apply)[0],2)
kn_pred=round(kn.predict(poly_apply)[0],2)
rf_pred=round(rf.predict(original_apply)[0],2)
print(f'决策树回归:{dt_pred}万元')
print(f'k近邻回归:{kn_pred}万元')
print(f'随机森林回归:{rf_pred}万元')

决策树回归:102.9万元
k近邻回归:122.71万元
随机森林回归:116.49万元


In [206]:
#利用加权平均法
#去平衡最终的结果
#w1=0.24（决策树）
#w2=0.24（K近邻）
#w3=0.52（随机森林）
final_pred = (
    0.24 * dt_pred + 
    0.24 * kn_pred + 
    0.52 * rf_pred
)
print(f'预测房价:{final_pred}万元')

预测房价:114.7212万元
